# Exercise - Sensors

<div class="alert alert-block alert-success">
This is an exercise in understanding the concept of error propagation and multi-sensor systems based on Chapter 3 “Sensors”.

## Content <a id="sec_toc"> </a>

[Read Sensor Datasheet](#sec_a)

[Sensor Uncertainty](#sec_b)

[Single-Sensor vs. Multi-Sensor System](#sec_c)


In [ ]:
! pip install matplotlib pandas numpy scipy

Import the needed libaries:

In [3]:
import math
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [4]:
# DO NOT CHANGE Plot functions
# Plot the true vs. ideal sensor function with linearity bounds
def plot_sensor_line(capacity, true, ideal, lowB, upB, linearity, title, xlim=None, ylim=None):
    plt.figure(figsize=(10, 6))
    plt.plot(capacity, true, color="blue", linewidth=2, label="True Senor Function")
    plt.plot(capacity, ideal, color="green", linestyle="-.", linewidth=1.5, label="Ideal Linear Function")
    plt.fill_between(capacity, lowB, upB, color="orange", alpha=0.3, label=f"Linearity Bound ±{linearity}% of Output")

    plt.xlabel("Meassured Force [N]")
    plt.ylabel("Sensor Output [mV]")
    plt.title(title)
    plt.legend(loc='upper left')
    if xlim != None:
        plt.xlim(xlim)
    if ylim != None:
        plt.ylim(ylim)
    plt.show()


def check_value(measuring_point):
    if(measuring_point == None):
        print("The measuring_point variable is still 'None'! Please adjust the variable. ")
        sys.exit()

## Read Sensor Datasheet<a id="sec_a"></a>

TASK: Read the uploaded Datasheet of the FSS015WNSX force sensor and fill out the blanks below with the corret values.

Note: For simplification we assume that g = 10m/s^2 for the following tasks and the questions in moodle. 

In [ ]:
min_force = ...... # minimal sensing force given in [N]          
max_force = ...... # maximal sensing force given in [N] 
typical_voltage = ..... # typical span given in [mV] (DC) 
typical_sensitivity = ...... # Typical sensitivity given in [mV/N]
typical_nullOffset = ..... # Typical Null offset given in [mV]
delta_sensitivity = ...... # sensitivity given in [mV/N]   
delta_nullOffset = ....... # Null Offset given in [mV]    
delta_linearity = ......  # Linearity given in [% span]   
delta_repeatability = ....... # Repetability given in [% span]

The next code block vizualizes the true vs. the ideal sensor line. The plot is based on the values extracted from the data sheet. Since we do not have true sensor data we will fit a polynomial line trough the start and end-point and the worst possible value for the middle-point.

In [ ]:
# DO NOT CHANGE (Plot sensor line)
# vector that holds all 500 plotting points
force_range = np.linspace(min_force, max_force, 500)

# compute ideal sensor curve (k*x + d)
ideal_sensor = typical_nullOffset + typical_sensitivity * force_range

middle_point = (max_force - min_force)/2
# Fixed meassuring points
points = np.array([
    [min_force, 0],                 # Start at (0 N, 0 mV)
    [middle_point, (typical_nullOffset + typical_sensitivity * middle_point) * (1-delta_linearity/100)],   # Midpoint at (75 N, k*75+d mV)
    [max_force, typical_voltage]  # Endpunkt (150 N, 360 mV)
])

# Calculate the coefficients of the quadratic polynomial passing through the three points
coefficients = np.polyfit(points[:, 0], points[:, 1], 2)
poly_sensor = np.poly1d(coefficients)
# Compute true sensor curve
true_sensor = poly_sensor(force_range)

# Bounds_based on linearity
linearity_bound_upper = ideal_sensor * (1+ delta_linearity/100)
linearity_bound_lower = ideal_sensor * (1- delta_linearity/100)

# Plot the true sensor line vs. the ideal sensor line
plot_sensor_line(force_range,true_sensor, ideal_sensor, linearity_bound_lower, linearity_bound_upper, delta_linearity,
                    "Comparison ideal vs. worst sensor function")

Since the error is very small, the next code snipped plots a zoomed in version of the plot above.

In [ ]:
# Plot the true sensor line vs. the ideal sensor line, Zoomed in Version
plot_sensor_line(force_range,true_sensor, ideal_sensor, linearity_bound_lower, linearity_bound_upper, delta_linearity,
                    "Comparison ideal vs. worst sensor function (Zoomed)", (7,8), (150,200))

[Back](#sec_toc)

## Sensor Uncertainty<a id="sec_b"></a>


In this chapter we look at the sensor uncertainty at a specific point. Therfore, we use two different approaches to calculate the uncertainty. First we need  to calculate the delta of the operating force that is constructed out of different parts of the sensor uncertaintys.

In [ ]:
# Calculating the deltas in Newton
delta_rep = delta_repeatability / 100 * max_force #[N]
delta_lin = delta_linearity / 100 * max_force # [N]

# Calculates all sensor uncertanties
adc_converter = 12 #[Bit]
delta_Vadc = typical_voltage /(2**adc_converter) # [mV]
delta_xadc = delta_Vadc / delta_sensitivity #[N]

delta_x = (delta_rep**2 + delta_lin**2 + delta_xadc**2)**(1/2)
print(f"Delta of the Force: {delta_x} N")

### Classical Error Propagation


Within the next code snipped the classical error propagation is used for the linear function of the ideal sensor.
$$\Delta_y = \frac{\delta_y}{\delta_k} * \Delta_k + \frac{\delta_y}{\delta_x} * \Delta_x + \frac{\delta_y}{\delta_d} * \Delta_d $$

In [ ]:
measuring_point = None # [N] CHANGE the measuring point within the sensor span: min_force to max_force 

####### DO NOT CHANGE #########
check_value(measuring_point)   
delta_k = delta_sensitivity
delta_d = delta_nullOffset
delta_y = measuring_point * delta_k + typical_sensitivity * delta_x + 1 * delta_d

print(f"Upper Bound for sensor uncertantiy at point {measuring_point}N: {delta_y} mV")

### Gaussian Error Propagation Law


Within the next code snipped the gaussian error propagation is used for the linear function of the ideal sensor. The law is used to calculate the upperbound of the standard deviation of the sensor uncertainty.
$$\sigma_y^2 = \sigma_x^2 + \sigma_k^2 + \sigma_d^2 $$

In [ ]:
# DO NOT CHANGE
sigma_s1 = measuring_point * delta_k
sigma_s2 = typical_sensitivity * delta_x
sigma_s3 = delta_d

sigma_y = ((sigma_s1)**2 + (sigma_s2)**2 + (sigma_s3)**2)**(1/2)
print(f"Standard deviation of sensor uncertainty at point {measuring_point}N: {sigma_y} mV")

[Back](#sec_toc)

## Single-Sensor vs. Multi-Sensor System <a id="sec_c"></a>

In this chapter, we are using a double sensor system to predict whether it will rain or not. \
This means that our model typically relies on two types of input data — temperature and humidity — to make predictions. \
We will have a look at our data in a scatter plot and we will also try to make predictions with either only temperature or humidity.

In [ ]:
# DO NOT CHANGE

def generate_weather_data(mean_temp_no_rain, std_temp_no_rain, mean_humid_no_rain, std_humid_no_rain,
                          mean_temp_rain, std_temp_rain, mean_humid_rain, std_humid_rain,
                          n_samples=1000, flip_prob=0.00, random_seed=None):
    """
    Generate weather data with specified means and standard deviations for temperature and humidity.

    Parameters:
    mean_temp_no_rain, std_temp_no_rain: Mean and std for temperature when no rain.
    mean_humid_no_rain, std_humid_no_rain: Mean and std for humidity when no rain.
    mean_temp_rain, std_temp_rain: Mean and std for temperature when rain.
    mean_humid_rain, std_humid_rain: Mean and std for humidity when rain.
    n_samples: Total number of samples to generate (split equally between rain and no rain).
    flip_prob: Probability of flipping the label (to introduce noise).
    random_seed: int, optional. Random seed for reproducibility (default: None).

    Returns:
    pd.DataFrame: DataFrame with columns ['temperature', 'humidity', 'label'].
    """
    if random_seed is not None:
        np.random.seed(random_seed)

    n_no_rain = n_samples // 2
    n_rain = n_samples - n_no_rain

    # Generate data for no rain
    temp_no_rain = np.random.normal(mean_temp_no_rain, std_temp_no_rain, n_no_rain)
    humid_no_rain = np.random.normal(mean_humid_no_rain, std_humid_no_rain, n_no_rain)
    labels_no_rain = np.zeros(n_no_rain)

    # Generate data for rain
    temp_rain = np.random.normal(mean_temp_rain, std_temp_rain, n_rain)
    humid_rain = np.random.normal(mean_humid_rain, std_humid_rain, n_rain)
    labels_rain = np.ones(n_rain)

    # Combine the data
    temp = np.concatenate([temp_no_rain, temp_rain])
    humid = np.concatenate([humid_no_rain, humid_rain])
    labels = np.concatenate([labels_no_rain, labels_rain])

    # Flip labels with the specified probability
    flip_indices = np.random.rand(n_samples) < flip_prob
    labels[flip_indices] = 1 - labels[flip_indices]

    # Create a DataFrame
    data = pd.DataFrame({
        'temperature': temp,
        'humidity': humid,
        'label': labels
    })

    return data

def plot_weather_data(data):
    """
    Plot the weather data as a scatter plot.

    Parameters:
    data: pd.DataFrame with columns ['temperature', 'humidity', 'label'].
    """
    colors = ['orange', 'blue']
    labels = ['No Rain', 'Rain']

    plt.figure()

    for label, color, label_name in zip([0, 1], colors, labels):
        subset = data[data['label'] == label]
        plt.scatter(subset['humidity'], subset['temperature'], label=label_name, alpha=0.6, c=color)

    plt.xlabel('Humidity (%)')
    plt.ylabel('Temperature (°C)')
    plt.title('Weather Data (Rain vs No Rain)')
    plt.legend(loc='best')
    plt.grid(True)
    plt.show()

def train_and_predict(data, features=None, random_seed=42):
    """
    Trains and evaluates a binary classification model using specified features (or all by default),
    and plots predictions.

    Parameters:
    data: pd.DataFrame with feature columns and a 'label' column.
    features: list, optional. The features to use for training (default: all columns except 'label').
    random_seed: int, optional. Random seed for reproducibility (default: 42).

    Returns:
    None
    """
    # If no specific features are provided, use all columns except 'label'
    if features is None:
        features = [col for col in data.columns if col != 'label']

    # Ensure the features exist in the dataset
    for feature in features:
        if feature not in data.columns:
            raise ValueError(f"Feature '{feature}' not found in the data.")

    # Define X (features) and y (label)
    X = data[features]
    y = data['label']

    # Perform an 80-20 train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.32423, random_state=random_seed)

    # Train a Random Forest Classifier
    model = RandomForestClassifier(random_state=random_seed)
    model.fit(X_train, y_train)

    # Predict on the test set
    y_pred = model.predict(X_test)

    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    feature_desc = ", ".join(features)

    # If only one feature is used, reconstruct missing features for plotting
    if len(features) == 1:
        missing_feature = 'temperature' if features[0] == 'humidity' else 'humidity'
        X_test[missing_feature] = data.loc[X_test.index, missing_feature]

    # Plot predictions with a detailed title including accuracy
    title = f"Predictions Using Features: {feature_desc} (Accuracy: {accuracy:.4f})"
    plot_predictions(data, X_test, y_test, y_pred, title=title)

def plot_predictions(data, X_test, y_test, y_pred, title="Prediction Results"):
    """
    Plot predictions with temperature on the y-axis and humidity on the x-axis.
    Correct predictions are shown with dots, and incorrect ones with crosses.

    Parameters:
    data: Original DataFrame with ['temperature', 'humidity', 'label'].
    X_test: Test features (DataFrame).
    y_test: True labels for the test data.
    y_pred: Predicted labels for the test data.
    title: Custom title for the plot.

    Returns:
    None
    """
    colors = ['orange', 'blue']
    labels = ['No Rain', 'Rain']

    # Convert y_test and y_pred to numpy arrays for compatibility
    y_test = np.array(y_test)
    y_pred = np.array(y_pred)

    # Ensure indices of X_test align with the boolean masks
    X_test = X_test.reset_index(drop=True)

    plt.figure(figsize=(8, 6))

    for label, color, label_name in zip([0, 1], colors, labels):
        # Filter correct predictions
        correct = (y_test == label) & (y_test == y_pred)
        plt.scatter(
            X_test.loc[correct, 'humidity'],  # Humidity on x-axis
            X_test.loc[correct, 'temperature'],  # Temperature on y-axis
            label=f"Correct {label_name}", c=color, alpha=0.6, marker='o'
        )

        # Filter incorrect predictions
        incorrect = (y_test == label) & (y_test != y_pred)
        plt.scatter(
            X_test.loc[incorrect, 'humidity'],  # Humidity on x-axis
            X_test.loc[incorrect, 'temperature'],  # Temperature on y-axis
            label=f"Incorrect {label_name}", c=color, alpha=0.6, marker='x'
        )

    plt.xlabel('Humidity (%)')
    plt.ylabel('Temperature (°C)')
    plt.title(title)
    plt.legend(loc='best')
    plt.grid(True)
    plt.show()

In the code snipped below you can determine how the data distribution will look like by asserting values to the variables \
which determine the mean and standard deviation of the data. The type of distribution is a gaussian normal distribution.

TASK: Change the values below to determine how the data distribution will look like.

In [ ]:
# Change the values (means and standard deviations) below to determine how the data distribution will look like
# Temperature in Degree Celsius [0, 70]     (Example: mean_temp_no_rain = 23.8)
# Humidity in % [0, 100]                    (Example: mean_humid_no_rain = 50)
mean_temp_no_rain = None
std_temp_no_rain = None
mean_humid_no_rain = None
std_humid_no_rain = None
mean_temp_rain = None
std_temp_rain = None
mean_humid_rain = None
std_humid_rain = None

data_graz = generate_weather_data(mean_temp_no_rain=mean_temp_no_rain, std_temp_no_rain=std_temp_no_rain,
                             mean_humid_no_rain=mean_humid_no_rain, std_humid_no_rain=std_humid_no_rain,
                             mean_temp_rain=mean_temp_rain, std_temp_rain=std_temp_rain,
                             mean_humid_rain=mean_humid_rain, std_humid_rain=std_humid_rain,
                             n_samples=1000, random_seed=43)
plot_weather_data(data_graz)

# Train and predict using only temperature
train_and_predict(data_graz, features=['temperature'], random_seed=42)

# Train and predict using only humidity
train_and_predict(data_graz, features=['humidity'], random_seed=42)

# Train and predict using all features
train_and_predict(data_graz, random_seed=42)

[Back](#sec_toc)